In [1]:
import pandas as pd 
import numpy as np 


In [2]:
import sklearn as sk 

In [3]:
import seaborn as sns 
import matplotlib.pyplot as plt 

In [18]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [11]:
y_train =  df_train.loc[:, 'Survived']

#y_test  = df_test.loc[:,'Survived']

# step 1: feature extraction 

In [5]:
#2: ticket is catagorical nominal; onehotencoding 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder

In [7]:

enc = OneHotEncoder(handle_unknown='ignore')


In [20]:
def processPipeline(df): 
    #drop name, ticket, cabin
    df.drop(columns = ['Name','Ticket','Cabin'], inplace = True)
    #transofrm embarked 
    
    df['Embarked'] = df['Embarked'].map(lambda x: str(x))
    
    df = df.loc[~(df.Embarked == 'nan'), :]
    #feature eng sex and embarked 
    #label encoder 
    labelenc = LabelEncoder()
    df['sex_cat'] = labelenc.fit_transform(df['Sex'])
    sex_df = pd.DataFrame(enc.fit_transform(df['sex_cat'].to_numpy().reshape(-1,1)).toarray()
                     ,columns= ['male','female'])
    
    df = df.join(sex_df)
    
    df['embarked_cat'] = labelenc.fit_transform(df['Embarked'])
    #onehotencoding 
  

    df_embarked = pd.DataFrame(enc.fit_transform(df['embarked_cat']
                                            .to_numpy().reshape(-1,1)).toarray(), 
                          columns= ['c','q','s'])
    
    df = df.join(df_embarked)
    
    #drop columns 
    df.drop(columns = ['embarked_cat','sex_cat','Sex','Embarked'], inplace = True)
    #drop missing values 
    #df.dropna(inplace = True)
    #index passengerID
    df.set_index ( 'PassengerId', inplace = True)
    
    
    return df 


In [19]:
df_train = processPipeline(df_train)


/Applications/anaconda3/envs/Projects/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [21]:
df_test = processPipeline(df_test)

In [22]:
#modeling start 
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler

In [23]:
#scaling 
pipe_lr = Pipeline([('std', StandardScaler()),
                    ('clf', LogisticRegressionCV(cv = 10, random_state=42))])
pipe_rf = Pipeline([('std', StandardScaler()),
                    ('clf',RandomForestClassifier(random_state=42))])
pipe_svc = Pipeline([('std', StandardScaler()),
                     ('clf', SVC(random_state=42))])

In [24]:
#cv 
from sklearn.model_selection import GridSearchCV

In [25]:
param_range = [x for x in range(1,11)]
grid_lr  = {
    'clf__penalty': ['l1','l2'], 
    'clf__solver': ['liblinear']
}

grid_rf = {
    'clf__min_samples_leaf': param_range,
    'clf__criterion': ['gini','entropy'],
    'clf__min_samples_split': param_range[1:], 
    'clf__max_depth': param_range
}

grid_svc = {
    'clf__kernel': ['linear','rbf'], 
    'clf__C': param_range
}


In [26]:
#final models 
gs_lr = GridSearchCV(estimator=pipe_lr, param_grid=grid_lr,
                    scoring = 'accuracy',cv = 10)
gs_rf = GridSearchCV(estimator=pipe_rf, param_grid=grid_rf,
                    scoring = 'accuracy',cv = 10)
gs_svc = GridSearchCV(estimator=pipe_svc, param_grid=grid_svc,
                    scoring = 'accuracy',cv = 10)


In [27]:
y_train = df_train.loc[:,'Survived']
x_train = df_train.drop(columns = 'Survived')


In [28]:
gs_svc.fit(x_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('std',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        SVC(C=1.0, break_ties=False,
                                            cache_size=200, class_weight=None,
                                            coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='scale',
                                            kernel='rbf', max_iter=-1,
                                            probability=False, random_state=42,
                                            shrinking=True, tol=0.001,
                                            ve

In [29]:
gs_svc.best_score_

0.7253521126760564

In [35]:
from sklearn.impute import SimpleImputer 

In [42]:
df_test.dtypes

Pclass      int64
Age       float64
SibSp       int64
Parch       int64
Fare      float64
male      float64
female    float64
c         float64
q         float64
s         float64
dtype: object

In [59]:
impt = SimpleImputer()
df_test['Age'] = impt.fit_transform(df_test['Age'].to_numpy().reshape(-1,1))
df_test['Fare'] = impt.fit_transform(df_test['Fare'].to_numpy().reshape(-1,1))

In [61]:
df_test.isna().any()

Pclass    False
Age       False
SibSp     False
Parch     False
Fare      False
male      False
female    False
c         False
q         False
s         False
dtype: bool

In [62]:
predictions = gs_svc.predict(df_test)

In [63]:
predict_csv = pd.DataFrame({'PassengerId': df_test.index, 'Survived': predictions})
predict_csv.set_index('PassengerId', inplace = True)

In [64]:
predict_csv.to_csv('titanic_predictions.csv')

In [91]:
gs_svc.best_estimator_

Pipeline(memory=None,
         steps=[('std',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 SVC(C=9, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=42, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [92]:
svc_model = SVC(C = 9, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=42, shrinking=True,
                     tol=0.001, verbose=False)


In [98]:
svc_model.fit(x_train,y_train).score

<bound method ClassifierMixin.score of SVC(C=9, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)>

In [102]:
from sklearn.metrics import accuracy_score

In [ ]:
svc_model.

In [103]:
accuracy_score(y_train,svc_model.predict(x_train))

0.719718309859155